## Develop a Deep Learning Based Churn Prediction Engine


### Import required libraries

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

#### Load & understand the data 

In [2]:
# Read the data
data = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Look at a snapshot of data
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
data.shape

(7043, 21)

In [4]:
# See the summary stats and frequency distribution of features
data.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [5]:
# Lets' see if there are any missing values

print (data.apply(lambda x: sum(x.isnull()),axis=0))
print (np.where(data.applymap(lambda x: x == ' ')))

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64
(array([ 488,  753,  936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754]), array([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]))


In [6]:
# Let's see if there is class imbalance in the target variable
print (data['Churn'].value_counts(ascending=True))

Yes    1869
No     5174
Name: Churn, dtype: int64


In [7]:
data['TotalCharges'].replace(to_replace = ' ', value= np.nan, inplace = True)
data['TotalCharges'] = data['TotalCharges'].astype(float)
data.dropna(axis=0, inplace=True)

### Observations:

**On Type conversions:**

- Columns like CustomerID can be removed from the analysis
- We see that 'Tenure' and 'MonthlyCharges' are numeric columns present in the data, with the data close to normal distribution. 
- Along with them, 'TotalCharges' is also a numeric column but contains some info missing, but still is not a nan.
- The column 'SeniorCitizen' is a categorical column by its nature with 'Yes' as 1, and No as 0. So it shuold be converted into Categorical type
- All the categorical attribtues are strings. Hence there is need to convert them into numbers, by a way of encoding.
- Among the categorical attribtues, majority of them have binary classes(2 levels). Label encoding would help assign labels 0,1 for the levels as appropriate.
- But attributes like 'PaymentMethod', 'Contract', 'InternetService' are nominal and have more than 2 levels. So along with label encoding, we need to convert them into equidistant levels.

**On Missingness of data:**
 The data is clean and there are no missing values in the data
 
**On the class imbalance in the target attribute**
There are more instances where the customers din't churn than those that have custoemrs churned out. Class imbalance is clearly seen.

## Data Preprocessing

### Split the data into train and test sets


In [8]:
from sklearn.model_selection import train_test_split

y = data['Churn']
X = data.loc[:, data.columns != 'Churn']

X_train, X_test, y_train, y_test =   train_test_split(X, y, test_size=0.20, random_state=111)

print(X_train.shape, X_test.shape)

(5625, 20) (1407, 20)


In [9]:
### Remove customerID
X_train.drop(['customerID'], axis = 1, inplace=True)
X_test.drop(['customerID'], axis = 1, inplace=True)

### Missing values

In [10]:
# Impute missing values, if any!. Check number of missing values
print("Num missing values before imputation:")
print(pd.DataFrame(X_train['TotalCharges']).isnull().sum())

print("Num missing values before imputation:")
print(pd.DataFrame(X_test['TotalCharges']).isnull().sum())

Num missing values before imputation:
TotalCharges    0
dtype: int64
Num missing values before imputation:
TotalCharges    0
dtype: int64


### Type Conversions

In [11]:
# Convert 'SeniorCitizen' column into categorical
X_train['SeniorCitizen']=pd.Categorical(X_train['SeniorCitizen'])
X_test['SeniorCitizen']=pd.Categorical(X_test['SeniorCitizen'])

In [12]:
# Encode target variables to 0, 1
y_train = y_train.map(dict(Yes=1, No=0))
y_test = y_test.map(dict(Yes = 1, No=0))
print(y_train.shape, y_test.shape)

(5625,) (1407,)


### Standardizing numeric attributes

In [13]:
# Divide the columns into 3 categories, one ofor standardisation, one for label encoding and one for one hot encoding
num_cols = ["tenure", 'MonthlyCharges', 'TotalCharges']
cat_cols_ohe =['PaymentMethod', 'Contract', 'InternetService'] # those that need one-hot encoding
cat_cols_le = list(set(X_train.columns)- set(num_cols) - set(cat_cols_ohe)) #those that need label encoding

In [14]:
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()

X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])


### Encoding attributes

In [15]:
X_train = pd.DataFrame(X_train)
X_test= pd.DataFrame(X_test)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder()
ohe = OneHotEncoder()

for col in cat_cols_le:
    le.fit(X_train[col])
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])



In [16]:
print(X_train.shape, X_test.shape)

(5625, 19) (1407, 19)


In [17]:
ohe.fit(X_train[cat_cols_ohe])
tr_cols= ohe.transform(X_train[cat_cols_ohe])
te_cols = ohe.transform(X_test[cat_cols_ohe])

X_train.drop(columns=cat_cols_ohe, inplace=True)
X_test.drop(columns=cat_cols_ohe, inplace=True)

X_train = np.hstack((X_train,tr_cols.toarray()))
X_test = np.hstack((X_test, te_cols.toarray()))
print(X_train.shape, X_test.shape)




(5625, 26) (1407, 26)


In [18]:
print(y_train.shape, y_test.shape)

(5625,) (1407,)


In [19]:
print(y_train.value_counts(), '\n', y_test.value_counts())

0    4148
1    1477
Name: Churn, dtype: int64 
 0    1015
1     392
Name: Churn, dtype: int64


## Building the ANN Model

In [20]:
#Importing necessary modules
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.model_selection import train_test_split

In [21]:
seed = 7
np.random.seed(seed)

In [22]:

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

input_shape = X_train.shape[1]

In [23]:
model = Sequential()
model.add(Dense(32, input_dim=input_shape, kernel_initializer='uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))



model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.fit(X_train, y_train,
              epochs=25,
          batch_size=24, class_weight={0:0.2, 1:0.8})



Epoch 1/25
235/235 [==============================] - 1s 2ms/step - loss: 0.2281 - accuracy: 0.4923
Epoch 2/25
235/235 [==============================] - 0s 2ms/step - loss: 0.1807 - accuracy: 0.7157
Epoch 3/25
235/235 [==============================] - 0s 2ms/step - loss: 0.1790 - accuracy: 0.6910
Epoch 4/25
235/235 [==============================] - 0s 2ms/step - loss: 0.1769 - accuracy: 0.7090
Epoch 5/25
235/235 [==============================] - 0s 2ms/step - loss: 0.1771 - accuracy: 0.6860
Epoch 6/25
235/235 [==============================] - 0s 2ms/step - loss: 0.1686 - accuracy: 0.7119
Epoch 7/25
235/235 [==============================] - 0s 2ms/step - loss: 0.1686 - accuracy: 0.7133
Epoch 8/25
235/235 [==============================] - 0s 2ms/step - loss: 0.1748 - accuracy: 0.7177
Epoch 9/25
235/235 [==============================] - 0s 2ms/step - loss: 0.1725 - accuracy: 0.7145
Epoch 10/25
235/235 [==============================] - 0s 2ms/step - loss: 0.1619 - accuracy: 0.7182

In [24]:

score = model.evaluate(X_test, y_test, batch_size=20)

71/71 [==============================] - 0s 1ms/step - loss: 0.5723 - accuracy: 0.6986


In [25]:
print(score)
print ("Accuracy : %s" % "{0:.3%}".format(score[1]))


[0.5722663998603821, 0.6986495852470398]
Accuracy : 69.865%


In [26]:

train_pred_dl=model.predict_classes(X_train)
test_pred_dl=model.predict_classes(X_test)

In [27]:
from sklearn import metrics
mlp_conf_matrix = metrics.confusion_matrix(y_test, test_pred_dl)
print (mlp_conf_matrix)

[[644 371]
 [ 53 339]]


In [28]:

accuracy = metrics.accuracy_score(y_test,test_pred_dl)
    
print ("Accuracy : %s" % "{0:.3%}".format(accuracy))

#Print Recall
recall = metrics.recall_score(y_test,test_pred_dl)
    
print ("Recall : %s" % "{0:.3%}".format(recall))

Accuracy : 69.865%
Recall : 86.480%


## MLP using features from AutoEncoders

In [29]:
encoding_dim  = 32
# this is our input placeholder
input_img = Input(shape=(input_shape,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(input_shape, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(inputs=input_img, outputs=decoded)

In [30]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy') #optimizer = adam --- can also be used 

In [31]:
autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=24,
                shuffle=True,
                validation_data=(X_test, X_test))

Epoch 1/50
235/235 [==============================] - 1s 3ms/step - loss: 0.7873 - val_loss: 0.7839
Epoch 2/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7846 - val_loss: 0.7813
Epoch 3/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7811 - val_loss: 0.7787
Epoch 4/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7780 - val_loss: 0.7760
Epoch 5/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7750 - val_loss: 0.7732
Epoch 6/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7721 - val_loss: 0.7705
Epoch 7/50
235/235 [==============================] - 0s 1ms/step - loss: 0.7704 - val_loss: 0.7678
Epoch 8/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7668 - val_loss: 0.7651
Epoch 9/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7644 - val_loss: 0.7624
Epoch 10/50
235/235 [==============================] - 0s 2ms/step - loss: 0.7617 - val_loss: 0.7597

In [32]:
# this model maps an input to its encoded representation
encoder = Model(inputs=input_img, outputs=encoded)


In [33]:
x_train_encoded = encoder.predict(X_train)
x_test_encoded = encoder.predict(X_test)

In [34]:
x_test_encoded

array([[0.730636  , 0.        , 1.5105859 , ..., 0.88970137, 0.        ,
        1.3669732 ],
       [0.6253408 , 0.        , 2.4669418 , ..., 1.1438243 , 0.        ,
        0.07685003],
       [0.        , 0.702132  , 1.4488927 , ..., 1.5563151 , 0.        ,
        0.        ],
       ...,
       [0.71535593, 0.04081775, 1.6565399 , ..., 0.5480147 , 0.        ,
        0.10981148],
       [0.        , 0.24502505, 1.5094392 , ..., 2.099444  , 0.        ,
        0.        ],
       [0.        , 0.6316507 , 1.6074859 , ..., 0.923262  , 0.        ,
        0.75907373]], dtype=float32)

In [35]:
x_train_encoded.shape

(5625, 32)

In [36]:
x_test_encoded.shape

(1407, 32)

In [37]:
model2 = Sequential()

model2.add(Dense(64, input_dim = 32, kernel_initializer='uniform', activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(16, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

In [38]:
model2.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [39]:
x_train_encoded.shape

(5625, 32)

In [40]:
model2.fit(x_train_encoded, y_train, batch_size=32, epochs=25, class_weight={0:0.2, 1:0.8})


Epoch 1/25
176/176 [==============================] - 1s 2ms/step - loss: 0.2362 - accuracy: 0.4486
Epoch 2/25
176/176 [==============================] - 0s 2ms/step - loss: 0.1937 - accuracy: 0.6693
Epoch 3/25
176/176 [==============================] - 0s 2ms/step - loss: 0.1837 - accuracy: 0.6898
Epoch 4/25
176/176 [==============================] - 0s 2ms/step - loss: 0.1853 - accuracy: 0.6907
Epoch 5/25
176/176 [==============================] - 0s 2ms/step - loss: 0.1815 - accuracy: 0.6911
Epoch 6/25
176/176 [==============================] - 0s 2ms/step - loss: 0.1852 - accuracy: 0.6774
Epoch 7/25
176/176 [==============================] - 0s 2ms/step - loss: 0.1810 - accuracy: 0.6870
Epoch 8/25
176/176 [==============================] - 0s 2ms/step - loss: 0.1818 - accuracy: 0.6789
Epoch 9/25
176/176 [==============================] - 0s 2ms/step - loss: 0.1793 - accuracy: 0.6869
Epoch 10/25
176/176 [==============================] - 0s 2ms/step - loss: 0.1769 - accuracy: 0.6824

In [41]:
score2 = model2.evaluate(x_test_encoded, y_test)
print (score2)

44/44 [==============================] - 0s 1ms/step - loss: 0.5540 - accuracy: 0.6994
[0.5539807081222534, 0.6993603706359863]


In [42]:

train_pred_dlac=model2.predict_classes(x_train_encoded)
test_pred_dlac=model2.predict_classes(x_test_encoded)

In [43]:
dlac_conf_matrix = metrics.confusion_matrix(y_test, test_pred_dlac)
print (dlac_conf_matrix)

[[653 362]
 [ 61 331]]


In [44]:

accuracy = metrics.accuracy_score(y_test,test_pred_dlac)
    
print ("Accuracy : %s" % "{0:.3%}".format(accuracy))

#Print Recall
recall = metrics.recall_score(y_test,test_pred_dlac)
    
print ("Recall : %s" % "{0:.3%}".format(recall))

Accuracy : 69.936%
Recall : 84.439%
